In [3]:
import pandas as pd
import numpy as np
from scipy.special import softmax
import copy
import json
from tqdm import tqdm_notebook
import pickle
from sklearn.metrics.cluster import adjusted_mutual_info_score

In [4]:
meta = pd.read_csv('title_tags_description/title_tags_description_meta.csv', lineterminator='^')

# Read index to word json file
### Warning: the keys (word indices) are string type
with open('title_tags_description/title_tags_description_indexword.json') as json_file:
    vocab = json.load(json_file)
    
# Read BoW file
bows = np.load('title_tags_description/title_tags_description_bow.npy')

In [5]:
vocab = list(vocab.values())

In [7]:
bow = bows

In [8]:
corpus = []
for i in range(len(bow)):
    doc_corpus = []
    for ind in list(np.where(bow[i]>0)[0]):
        doc_corpus.append((ind, bow[i][ind]))
    corpus.append(doc_corpus)

In [11]:
dictionary = {i:vocab[i] for i in range(len(vocab))}

In [31]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 16
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.

model = LdaModel(
    corpus=corpus,
    num_topics=num_topics,
    eval_every=eval_every
)

In [33]:
temp = model.get_document_topics(corpus)

In [35]:
def extract_topics():
    res = []
    for i in range(len(temp)):
        res.append(sorted(temp[i], key = lambda x: -x[1])[0][0])
    return res

In [36]:
topics = extract_topics()

In [37]:
# calculate Adjusted Mutual Information (AMI)

# clust_asgn and ground_truth are corresponding
def calculate_ami(clust_asgn, true_genres):
    #pred_clusters = np.concatenate(clust_asgn).ravel()
    return adjusted_mutual_info_score(true_genres, clust_asgn)

In [38]:
meta_genres = pd.read_csv('title_tags_description' + '/' + 'title_tags_description' + '_meta.csv', lineterminator='^', \
                      dtype={'genre':'category'}).genre.cat.codes

In [39]:
calculate_ami(topics, meta_genres)

0.18654357214580636